# Data Cleaning

## Import Libraries

In [24]:
# Modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numerize import numerize
import geopandas as gpd

## Import Dataframe

In [25]:
# Import dataframe
kp = pd.read_csv('https://raw.githubusercontent.com/Fery-K/Capstone_Project_TETRIS/master/Datasets/kredit_provinsi.csv')
k_mk = pd.read_csv('https://raw.githubusercontent.com/Fery-K/Capstone_Project_TETRIS/master/Datasets/kredit_modal_kerja_bank.csv')
k_i = pd.read_csv('https://raw.githubusercontent.com/Fery-K/Capstone_Project_TETRIS/master/Datasets/kredit_investasi_bank.csv')
k_k = pd.read_csv('https://raw.githubusercontent.com/Fery-K/Capstone_Project_TETRIS/master/Datasets/kredit_konsumsi_bank.csv')
pdrb = pd.read_csv('https://raw.githubusercontent.com/Fery-K/Capstone_Project_TETRIS/master/Datasets/pdrb.csv')

## Check Missing Value

In [26]:
print(kp.isna().sum())
print(k_mk.isna().sum())
print(k_i.isna().sum())
print(k_k.isna().sum())
print(pdrb.isna().sum())

no          0
provinsi    0
tahun       0
kredit      0
dtype: int64
bank      0
tahun     0
kredit    0
dtype: int64
bank      0
tahun     0
kredit    0
dtype: int64
bank      0
tahun     0
kredit    0
dtype: int64
no            0
provinsi      0
tahun_2011    0
tahun_2012    0
tahun_2013    0
tahun_2014    0
tahun_2015    0
tahun_2016    0
tahun_2017    0
tahun_2018    0
tahun_2019    0
tahun_2020    0
tahun_2021    0
tahun_2022    0
dtype: int64


## Detect Missing Value Location

In [27]:
col_pdrb = pdrb.columns.values.tolist()
for i in range(len(col_pdrb)):
    if pdrb[col_pdrb[i]].isna().sum() == 1:
        for j in range(pdrb.shape[0]):
            if pdrb[col_pdrb[i]].isna().loc[j]:
                print(f"Data hilang:\n[tahun,provinsi]: {[col_pdrb[i],pdrb['provinsi'].loc[j]]}")

## Impute Missing Value

In [28]:
pdrb.loc[33, 'tahun_2011'], pdrb.loc[33, 'tahun_2012'], pdrb.loc[33, 'tahun_2013'] = 0, 0, 0
pdrb.loc[34, 'tahun_2015'] = pdrb.transpose()[34].loc[col_pdrb[2:]].mean()

TypeError: Could not convert 30?112,3731?519,9332?874,7634?127,7235?161,8936?468,6237?851,3739?340,5641?021,6139?778,6840?780,3142?463,78 to numeric

# Data Exploration and Visualization

In [ ]:
# Lookup dataframe
print(kp.head())
print(pdrb.head())
print(k_k.head())
print(k_i.head())
print(k_mk.head())

## Visualisasi data kredit menurut provinsi

In [ ]:
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = 6, 10

kp_pivot = kp.pivot(index='provinsi', columns='tahun', values='kredit').reset_index()

kp_pivot.plot(
    x='provinsi',
    kind='barh',
    stacked=True,
    colormap='tab20c_r'
)

plt.title('Jumlah Rata-Rata Kredit per Tahun Tiap Provinsi')
plt.xlabel('Jumlah Kredit (Milyar Rupiah)')
plt.ylabel('')
plt.xscale('log')
plt.show()

## Visualisasi Hubungan Kredit dan PDRB

In [ ]:
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = 12, 6

dg = gpd.read_file('https://github.com/Fery-K/Capstone_Project_TETRIS/raw/master/Provinsi%20Shapefiles/Batas%20Provinsi.shp')
pdrb['provinsi'] = pdrb['provinsi'].str.upper()    
dg['Provinsi'] = dg['Provinsi'].replace(['KEPULAUAN BANGKA BELITUNG', 'DKI JAKARTA', 'DAERAH ISTIMEWA YOGYAKARTA'],
                                       ['BANGKA BELITUNG', 'D.K.I. JAKARTA', 'D.I. YOGYAKARTA'])

dg_pdrb = pd.merge(dg, pdrb, left_on='Provinsi', right_on='provinsi')

# # dg.plot(color='lightgreen', edgecolor='black')
# dg_pdrb.plot(column='tahun_2011', cmap='summer_r', legend=True, legend_kwds={'shrink': 0.6})
dg_pdrb['rerata'] = dg_pdrb.loc[:, 'tahun_2011':'tahun_2022'].mean(axis=1)
dg_pdrb.plot(column='rerata', cmap='summer_r', legend=True, legend_kwds={'shrink': 0.6})
plt.title('Persebaran Laju Indeks Pertumbuhan PDRB Pada Tiap Provinsi')
plt.xticks([])
plt.yticks([])
plt.ylabel('Laju Indeks Pertumbuhan PDRB', labelpad=-565)
plt.show()

In [ ]:
pdrb.columns = ['no', 'provinsi'] + [(2011+i) for i in range(12)]

pdrb_ver = pd.melt(pdrb, id_vars=['no', 'provinsi'], 
                   value_vars=[(2011+i) for i in range(12)],
                   var_name = 'tahun',
                   value_name = 'PDRB'
                  )
kp['provinsi'] = kp['provinsi'].str.upper()

kp_pdrb = pd.merge(kp[kp['tahun'] >= 2011].reset_index(), pdrb_ver[pdrb_ver['provinsi'] != 'NASIONAL'], on=['provinsi', 'tahun'], how='outer')

sns.regplot(data=kp_pdrb[kp_pdrb['provinsi'] == 'RIAU'], x='kredit', y='PDRB')
plt.yscale('linear')
plt.show()


## Visualisasi Kredit Berdasarkan Jenis Penggunaan

In [ ]:
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = 12, 6

k_k_tahun = k_k.groupby('tahun').sum()
k_i_tahun = k_i.groupby('tahun').sum()
k_mk_tahun = k_mk.groupby('tahun').sum()
k_k_tahun['jenis'] = ['konsumsi' for i in range(k_k_tahun.shape[0])]
k_i_tahun['jenis'] = ['investasi' for j in range(k_i_tahun.shape[0])]
k_mk_tahun['jenis'] = ['modal kerja' for k in range(k_mk_tahun.shape[0])]

k_jenis = pd.concat([k_k_tahun, k_i_tahun, k_mk_tahun]).reset_index()
k_pivot = k_jenis.pivot(index='tahun', columns='jenis', values='kredit').reset_index()

k_pivot.plot(
    x='tahun',
    kind='bar',
    stacked=True,
    colormap='tab20c_r'
)

x_ax = [0.5*i*(10**6) for i in range(1, 11)]

plt.title('Jumlah Rata-Rata Kredit per Tahun Berdasarkan Jenis Penggunaan')
plt.xticks(rotation=0)
plt.yticks(x_ax, [numerize.numerize(n) for n in x_ax])
plt.ylabel('Jumlah Kredit (Milyar Rupiah)')
plt.xlabel('Tahun')
plt.show()